In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
class maskedSelfAttention(nn.Module):
    def __init__(self,d_model=2,row_dim=0,col_dim=1):
        super().__init__()
        self.W_q=nn.Linear(in_features=d_model,out_features=d_model,bias=False)
        self.W_k=nn.Linear(in_features=d_model,out_features=d_model,bias=False)
        self.W_v=nn.Linear(in_features=d_model,out_features=d_model,bias=False)
        self.row_dim=row_dim
        self.col_dim=col_dim
    def forward(self,token_encodings,mask=None):
        q=self.W_q(token_encodings)
        k=self.W_k(token_encodings)
        v=self.W_v(token_encodings)
        sims=torch.matmul(q,k.transpose(dim0=self.row_dim,dim1=self.col_dim))
        scaled_sims=sims/torch.tensor(k.size(self.col_dim)**0.5)
        if mask is not None:
            scaled_sims=scaled_sims.masked_fill(mask=mask,value=-1e9)
        attention_percents=F.softmax(scaled_sims,dim=self.col_dim)
        attention_scores=torch.matmul(attention_percents,v)
        return attention_scores
        
            

In [12]:
encodings_matrix=torch.tensor([[1.16,0.23],
                               [0.57,1.36],
                               [4.41,-2.16]])
torch.manual_seed(42)
maskedSelfAttention=maskedSelfAttention(d_model=2,row_dim=0,col_dim=1)
mask=torch.tril(torch.ones(3,3))

In [13]:
mask=mask==0
mask

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

In [14]:
maskedSelfAttention(encodings_matrix,mask)

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)

In [16]:
maskedSelfAttention.W_q.weight.transpose(0,1)

tensor([[ 0.5406, -0.1657],
        [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>)

In [17]:
maskedSelfAttention.W_v(encodings_matrix)

tensor([[ 0.6038,  0.7434],
        [-0.3502,  0.5303],
        [ 3.8695,  2.4246]], grad_fn=<MmBackward0>)